# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [2]:
TRAINING_DATA_FILE = 'C:/Users/rachi/Downloads/train-data1.txt'
TEST_DATA_FILE = 'C:/Users/rachi/Downloads/test-data1.txt'

VOCAB_SIZE = 2500

# Read and Load Features from .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype = int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter = ' ', dtype = int)

In [5]:
sparse_train_data[:5]

array([[  0,   2,   0,   1],
       [  0,   7,   0,   1],
       [  0,  58,   0,   1],
       [  0, 273,   0,   1],
       [  0, 459,   0,   1]])

In [6]:
sparse_train_data[-5:]

array([[5570,  898,    0,    1],
       [5570, 1043,    0,    1],
       [5570, 1695,    0,    1],
       [5571,  161,    1,    1],
       [5571,  482,    1,    1]])

In [7]:
print('Number of rows in training file: ', sparse_train_data.shape[0])
print('Number of rows in test file: ', sparse_test_data.shape[0])

Number of rows in training file:  21766
Number of rows in test file:  9159


In [8]:
print('Number of emails in training file:', np.unique(sparse_train_data[:,0]).size)

Number of emails in training file: 3866


In [9]:
print('Number of emails in test file:', np.unique(sparse_test_data[:,0]).size)

Number of emails in test file: 1659


# Create a Empty DataFrame

In [10]:
column_names=['DOC_ID']+['CATEGORY']+list(range(0,VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5568, 5570, 5571])

In [13]:
full_train_data = pd.DataFrame(index = index_names, columns = column_names)
full_train_data.fillna(value=0, inplace = True)

C:\Users\rachi\AppData\Local\Temp\ipykernel_16380\2370846799.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_train_data.fillna(value=0, inplace = True)


In [14]:
full_train_data.head()

DOC_ID  CATEGORY  0  1  2  3  4  5  6  7  ...  2490  2491  2492  2493  \
0       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
1       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
2       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
3       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
4       0         0  0  0  0  0  0  0  0  0  ...     0     0     0     0   

   2494  2495  2496  2497  2498  2499  
0     0     0     0     0     0     0  
1     0     0     0     0     0     0  
2     0     0     0     0     0     0  
3     0     0     0     0     0     0  
4     0     0     0     0     0     0  

[5 rows x 2502 columns]

# Create a Full Matrix from a Sparse Matrix

In [15]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index = doc_id_names, columns = column_names)
    full_matrix.fillna(value=0, inplace = True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace = True)
    return full_matrix

In [16]:
%%time

full_train_data=make_full_matrix(sparse_train_data, VOCAB_SIZE)

C:\Users\rachi\AppData\Local\Temp\ipykernel_16380\630567689.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_matrix.fillna(value=0, inplace = True)


CPU times: total: 3.78 s
Wall time: 5.5 s


In [17]:
full_train_data.head()

CATEGORY  0  1  2  3  4  5  6  7  8  ...  2490  2491  2492  2493  \
DOC_ID                                       ...                           
0              0  0  0  1  0  0  0  0  1  0  ...     0     0     0     0   
1              0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
2              0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
3              0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   
4              1  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   

        2494  2495  2496  2497  2498  2499  
DOC_ID                                      
0          0     0     0     0     0     0  
1          0     0     0     0     0     0  
2          0     0     0     0     0     0  
3          0     0     0     0     0     0  
4          0     0     0     0     0     0  

[5 rows x 2501 columns]

In [18]:
full_train_data.tail()

CATEGORY  0  1  2  3  4  5  6  7  8  ...  2490  2491  2492  2493  \
DOC_ID                                       ...                           
5564           0  0  1  0  0  0  0  0  0  0  ...     0     0     0     0   
5566           1  1  1  0  0  0  0  0  0  0  ...     0     0     0     0   
5568           0  0  0  1  0  0  0  0  0  0  ...     0     0     0     0   
5570           0  0  0  0  0  0  1  0  0  0  ...     0     0     0     0   
5571           1  0  0  0  0  0  0  0  0  0  ...     0     0     0     0   

        2494  2495  2496  2497  2498  2499  
DOC_ID                                      
5564       0     0     0     0     0     0  
5566       0     0     0     0     0     0  
5568       0     0     0     0     0     0  
5570       0     0     0     0     0     0  
5571       0     0     0     0     0     0  

[5 rows x 2501 columns]

# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [19]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is: ', prob_spam)

Probability of spam is:  0.2397827211588205


# Total Number of  Words/Tokens

In [20]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

0     1     2     3     4     5     6     7     8     9     ...  2490  \
DOC_ID                                                              ...         
0          0     0     1     0     0     0     0     1     0     0  ...     0   
1          0     0     0     0     0     0     0     0     0     0  ...     0   
2          0     0     0     0     0     0     0     0     0     1  ...     0   
3          0     0     0     0     0     0     0     0     0     0  ...     0   
4          0     0     0     0     0     0     0     0     0     0  ...     0   

        2491  2492  2493  2494  2495  2496  2497  2498  2499  
DOC_ID                                                        
0          0     0     0     0     0     0     0     0     0  
1          0     0     0     0     0     0     0     0     0  
2          0     0     0     0     0     0     0     0     0  
3          0     0     0     0     0     0     0     0     0  
4          0     0     0     0     0     0     0     0     0  

[5 rows x 2500 columns]

In [21]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(3866,)

In [22]:
email_lengths[:5]

DOC_ID
0     7
1     3
2    16
3     4
4     6
dtype: int64

In [23]:
total_wc = email_lengths.sum()
total_wc

np.int64(22783)

# Number of Tokens in Spam and Ham Emails

In [24]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(927,)

In [25]:
spam_wc = spam_lengths.sum()
spam_wc

np.int64(6730)

In [26]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2939,)

In [27]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

np.int64(16053)

In [28]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [29]:
spam_wc + nonspam_wc - total_wc

np.int64(0)

# Summing the Tokens occuring in Spam

In [30]:
full_train_features.shape

(3866, 2500)

In [31]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

0     1     2     3     4     5     6     7     8     9     ...  2490  \
DOC_ID                                                              ...         
4          0     0     0     0     0     0     0     0     0     0  ...     0   
5          0     0     0     0     0     1     0     0     0     0  ...     0   
9          1     0     0     0     0     0     0     0     0     1  ...     0   
11         0     0     0     0     0     0     0     0     0     0  ...     0   
27         0     0     0     0     0     0     0     0     0     0  ...     0   

        2491  2492  2493  2494  2495  2496  2497  2498  2499  
DOC_ID                                                        
4          0     0     0     0     0     0     0     0     0  
5          0     0     0     0     0     0     0     0     0  
9          0     0     0     0     0     0     0     0     0  
11         0     0     0     0     0     0     0     0     0  
27         0     0     0     0     0     0     0     0     0  

[5 rows x 2500 columns]

In [32]:
train_spam_tokens.tail()

0     1     2     3     4     5     6     7     8     9     ...  2490  \
DOC_ID                                                              ...         
5547       0     0     0     0     0     0     0     0     0     0  ...     0   
5550       0     1     0     0     0     0     0     0     0     0  ...     0   
5562       0     0     0     0     0     0     0     0     0     0  ...     0   
5566       1     1     0     0     0     0     0     0     0     1  ...     0   
5571       0     0     0     0     0     0     0     0     0     0  ...     0   

        2491  2492  2493  2494  2495  2496  2497  2498  2499  
DOC_ID                                                        
5547       0     0     0     0     0     0     0     0     0  
5550       0     0     0     0     0     0     0     0     0  
5562       0     0     0     0     0     0     0     0     0  
5566       0     0     0     0     0     0     0     0     0  
5571       0     0     0     0     0     0     0     0     0  

[5 rows x 2500 columns]

In [33]:
train_spam_tokens.shape

(927, 2500)

In [34]:
summed_spam_tokens = train_spam_tokens.sum(axis = 0) + 1
summed_spam_tokens.shape

(2500,)

In [35]:
summed_spam_tokens.tail()

2495    1
2496    1
2497    1
2498    2
2499    1
dtype: int64

# Summing the Tokens occuring in Ham

In [36]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [37]:
summed_ham_tokens.shape

(2500,)

In [38]:
summed_ham_tokens.tail()

2495    1
2496    1
2497    3
2498    2
2499    3
dtype: int64

In [39]:
train_ham_tokens[2499].sum() + 1

np.int64(3)

# P(Token | Spam) - Probability that a Token occurs given that the Email is Spam

In [40]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.026219
1    0.009859
2    0.005417
3    0.010401
4    0.003467
dtype: float64

In [41]:
prob_tokens_spam.sum()

np.float64(0.9999999999999998)

# P(Token | Ham) - Probability that a Token occurs given that the Email is Non-Spam

In [45]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

np.float64(1.0)

# P(Token) - Probability that Token occurs

In [46]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all.sum()

np.float64(1.0)

# Save the Trained Model

In [47]:
TOKEN_SPAM_PROB_FILE = "token_spam_prob1.txt"
TOKEN_HAM_PROB_FILE = "token_ham_prob1.txt"
TOKEN_ALL_PROB_FILE = "token_all_prob1.txt"

np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [48]:
sparse_test_data.shape

(9159, 4)

In [49]:
%%time

full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

C:\Users\rachi\AppData\Local\Temp\ipykernel_16380\630567689.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_matrix.fillna(value=0, inplace = True)


CPU times: total: 1.58 s
Wall time: 2.45 s


In [50]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [51]:
TEST_FEATURE_MATRIX = 'test-features1.txt'
TEST_TARGET_FILE = 'test-target1.txt'

np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)